In [2]:
import os
import sys  
sys.path.insert(0, '..')


import tensorflow as tf
import utillity as utill
import yolo_utils
import numpy as np
import models as mdl
from tensorflow.keras import models, layers, activations as act

2022-01-31 20:34:53.371221: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-31 20:34:53.371259: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
config = {
  'image_size' : (608, 608, 3),
  'anchors' : [5, 8, 15, 28, 37, 61, 80, 125, 159, 230],
  'stride' : 32,
  'xyscale': 1.05,

  # Training
  'iou_loss_thresh': 0.5,
  'batch_size': 8,
  'num_gpu': 1,  # 2,

  # Inference
  'max_boxes': 100,
  'iou_threshold': 0.5,
  'score_threshold': 0.5,
}

In [3]:
model_path = os.path.join('..','assets', 'model.tflite')
weight_path = os.path.join('..', 'assets', 'new2-model-mobilenet-yolo-resume.159-107.74.h5')

## Create tf.lite model

In [4]:
def original_model(config, number_class):
  #anchors = np.array(config['anchors']).reshape((5, 2))

  inputs = layers.Input(config['image_size'])
  backbone = mdl.MobileNet(inputs)
  head = yolo_utils.yolo_head(backbone.outputs[2], number_of_class=number_class, anchors_size=5)
  # head = layers.Reshape((head.shape[1], head.shape[1], anchors.shape[0], 5+number_class))(head)
  # # spliter = tf.split(head, (2, 2, 1, number_class), axis=-1)
  # pred_xy = act.sigmoid(head[...,:2])
  # pred_conf = act.sigmoid(head[...,4:5])
  # pred_class = act.sigmoid(head[...,5:])

  # grid = np.meshgrid(np.arange(head.shape[1]), np.arange(head.shape[1]))  # (52, 52) (52, 52)
  # grid = np.expand_dims(np.stack(grid, axis=-1), axis=2)  # (52, 52, 1, 2)
  # grid = np.array(grid).astype('float32')

  # pred_xy = ((pred_xy * config['xyscale']) - 0.5 * (config['xyscale'] - 1) + grid) * config['stride']
  # pred_wh = act.exponential(head[...,2:4]) * anchors

  # box_x1y1 = pred_xy - pred_wh / 2  # (?, 52, 52, 3, 2)
  # box_x2y2 = pred_xy + pred_wh / 2
  # pred_x1y1x2y2 = np.concatenate((box_x1y1, box_x2y2), axis=-1)

  # predict = np.concatenate((pred_x1y1x2y2, pred_conf, pred_class), axis=-1)

  # # bs = tf.shape(pred_x1y1x2y2)[0]
  # # boxes = tf.zeros((bs, 0, 4))
  # # confidence = tf.zeros((bs, 0, 1))
  # # class_probabilities = tf.zeros((bs, 0, number_class))

  # # boxes = tf.concat([boxes, tf.reshape(pred_x1y1x2y2, (bs, -1, 4))], axis=1)
  # # confidence = tf.concat([confidence, tf.reshape(spliter[2], (bs, -1, 1))], axis=1)
  # # class_probabilities = tf.concat([class_probabilities, tf.reshape(spliter[3], (bs, -1, number_class))], axis=1)

  # # scores = confidence * class_probabilities
  # # boxes = tf.expand_dims(boxes, axis=-2)
  # # boxes = boxes / config['image_size'][0] 

  # # result = tf.image.combined_non_max_suppression(
  # #       boxes=boxes,  # y1x1, y2x2 [0~1]
  # #       scores=scores,
  # #       max_output_size_per_class=config['max_boxes'],
  # #       max_total_size=config['max_boxes'],  # max_boxes: Maximum nmsed_boxes in a single img.
  # #       iou_threshold=config['iou_threshold'],  # iou_threshold: Minimum overlap that counts as a valid detection.
  # #       score_threshold=config['score_threshold'],  # # Minimum confidence that counts as a valid detection.
  # #   )
  # # print(result[0].shape)
  # # print(result[1].shape)
  # # print(predict.shape)

 
  return models.Model(inputs, head)

In [5]:
new_model = original_model(config, 1)
new_model.load_weights(weight_path)

2021-12-15 16:09:11.649818: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-15 16:09:11.649877: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ozmonday): /proc/driver/nvidia/version does not exist
2021-12-15 16:09:11.650224: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 76, 76, 256)
(None, 38, 38, 512)
(None, 19, 19, 1024)


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = converter.convert()

2021-12-15 16:09:39.089705: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpxn3wljat/assets


2021-12-15 16:09:47.929982: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-12-15 16:09:47.930255: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-12-15 16:09:47.973143: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.611ms.
  function_optimizer: function_optimizer did nothing. time = 0.003ms.

2021-12-15 16:09:49.010221: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-12-15 16:09:49.010299: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-12-15 16:09:49.064098: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:210] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


In [17]:
with open('../assets/model.tflite', 'wb') as f:
  f.write(tflite_model)


In [3]:
interpreter = tf.lite.Interpreter(model_path='../assets/model1.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [9]:
print(output_details[0]['shape'])
print(output_details[1]['shape'])
print(output_details[2]['shape'])

[ 1 52 52 18]
[ 1 26 26 18]
[ 1 13 13 18]


In [10]:
img_ori = utill.open_image('../test_img/53_Raid_policeraid_53_189.jpg')
img = tf.image.resize(img_ori, (input_details[0]['shape'][1], input_details[0]['shape'][1]))
img = img / 255
img_exp = np.expand_dims(img, axis=0)
img_exp.shape

2021-12-05 20:33:35.279771: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-05 20:33:35.279817: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ozmonday): /proc/driver/nvidia/version does not exist
2021-12-05 20:33:35.280142: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(1, 416, 416, 3)

In [20]:
interpreter.set_tensor(input_details[0]['index'], img_exp)
interpreter.invoke()

anchors = np.array(config['anchors']).reshape((3, 3, 2))
large_output = interpreter.get_tensor(output_details[0]['index'])
medium_output = interpreter.get_tensor(output_details[1]['index'])
small_output = interpreter.get_tensor(output_details[2]['index'])
outputs = [large_output, medium_output, small_output]
outputs = yolo_utils.yolo_detector(outputs, anchors, 1, config['strides'], config['xyscale'])
outputs = utill.nms(outputs, config['image_size'], 1, config['iou_threshold'], config['score_threshold'])
# output_data = yolo_utils.get_boxes(output_data, anchors, 1, config['stride'], config['xyscale'])
# output_data = utill.nms(output_data, config['image_size'], 1, config['iou_threshold'], config['score_threshold'])
boxes = utill.get_detection_data(outputs, img_ori.shape)
#utill.plot_bbox(img_ori, boxes, True)

nms iou: 0.413 score: 0.3
# of bboxes: 1


In [ ]:
from tensorflow.keras import layers


inputs = layers.Input()
resize = tf.image.resize(inputs, (30, 30))


In [7]:
randoms = np.random.rand(12, 59, 59 ,5, 6)
randoms.shape

spliter = tf.split(randoms, (2,2,1,1), axis=-1)

spliter[0] = tf.sigmoid(spliter[0])
spliter[2] = tf.sigmoid(spliter[2])
spliter[3] = tf.sigmoid(spliter[3])
pred_xywh = tf.concat((spliter[0], spliter[1]), axis=-1)
print(pred_xywh.shape)

rang = np.arange(12)
rang = -np.sort(-rang).reshape(2,6)
print(rang)

(12, 59, 59, 5, 4)
[[11 10  9  8  7  6]
 [ 5  4  3  2  1  0]]
